## 加载数据 

In [91]:
import torch
import torch.nn as nn

In [92]:
import torchvision as tv
import torchvision.transforms as transforms
import torch.utils.data

In [93]:
show = tv.transforms.ToPILImage()  # 将Tensor转化为Image
# 定义对数据的预处理
transform = transforms.Compose([transforms.ToTensor(),  # 转为Tensor
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 归一化
                                ])
# 训练集
train_set = tv.datasets.CIFAR10(root='E:\Data\dataSet\cifar-10-python',
                                train=True,
                                download=True,
                                transform=transform)

train_loader =torch.utils.data.DataLoader(train_set,
                                          batch_size=4,
                                          shuffle=True)
#                                           num_workers=1)

# 测试集
test_set = tv.datasets.CIFAR10(root='E:\Data\dataSet\cifar-10-python',
                               train=False,
                               download=True,
                               transform=transform)
  
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=4,
                                          shuffle=False)
#                                           num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 模型结构

In [64]:
class Net(nn.Module):
    """输入为3通道的网络"""
    def __init__(self):
        super(Net, self).__init__()
        self.cba1 = nn.Sequential(
                    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
                    nn.BatchNorm2d(32),
                    nn.LeakyReLU()
                    )
        self.cba2 = nn.Sequential(
                    nn.Conv2d(32, 32, 3, 2, 1),
                    nn.BatchNorm2d(32),
                    nn.LeakyReLU()
                    )
        self.cba3 = nn.Sequential(
                    nn.Conv2d(32, 64, 3, 1, 1),
                    nn.BatchNorm2d(64),
                    nn.LeakyReLU()
                    )
        self.cba4 = nn.Sequential(
                    nn.Conv2d(64, 64, 3, 2, 1),
                    nn.BatchNorm2d(64),
                    nn.LeakyReLU()
                    )
        
        
#         self.features = nn.ModuleList([self.cba1, self.cba2, self.cba3, self.cba4])
        
        self.classifier = nn.Sequential(
                    nn.Linear(4096, 128),  # 规定尺寸大小的输入
                    nn.BatchNorm1d(128),
                    nn.LeakyReLU(),
                    nn.Dropout(0.5),
                    nn.Linear(128, 10),
                    nn.Softmax(dim=1)
                    ) 
        
    def forward(self, x):
#         x = self.features(x)  # 会报错，Module_List没有实现forward()方法
#         for model in self.features:
#              x = model(x)
        x = self.cba1(x)
        x = self.cba2(x)
        x = self.cba3(x)
        x = self.cba4(x)
        
        x = x.view(x.size()[0], -1)  # 按照Batch形式排列成[Batch, C*W*H]
#         x = nn.Linear(in_features=x.size()[-1], out_features=128)(x)  # 将模型放到cuda上时，在forward()中初始化层会报错；cpu上没问题
        x = self.classifier(x)
        return x

In [65]:
net = Net()

### 将模型数据放入cuda 

In [66]:
next(net.parameters()).device  # 判断模型位置

device(type='cpu')

In [67]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda')
net = net.to(device)  # 整个式子等价与net.cuda()

In [68]:
device

device(type='cuda')

In [69]:
next(net.parameters()).device

device(type='cuda', index=0)

In [70]:
input_data = torch.randn(4, 3, 32, 32)   # 判断数据位置

In [71]:
input_data.device

device(type='cpu')

In [72]:
input_data = input_data.cuda()

In [73]:
input_data.device

device(type='cuda', index=0)

### 使用torchsummay.summary展示层结构 

In [74]:
from torchsummary import summary

In [75]:
for name, p in net.named_parameters():
    print(name, p.size())

cba1.0.weight torch.Size([32, 3, 3, 3])
cba1.0.bias torch.Size([32])
cba1.1.weight torch.Size([32])
cba1.1.bias torch.Size([32])
cba2.0.weight torch.Size([32, 32, 3, 3])
cba2.0.bias torch.Size([32])
cba2.1.weight torch.Size([32])
cba2.1.bias torch.Size([32])
cba3.0.weight torch.Size([64, 32, 3, 3])
cba3.0.bias torch.Size([64])
cba3.1.weight torch.Size([64])
cba3.1.bias torch.Size([64])
cba4.0.weight torch.Size([64, 64, 3, 3])
cba4.0.bias torch.Size([64])
cba4.1.weight torch.Size([64])
cba4.1.bias torch.Size([64])
classifier.0.weight torch.Size([128, 4096])
classifier.0.bias torch.Size([128])
classifier.1.weight torch.Size([128])
classifier.1.bias torch.Size([128])
classifier.4.weight torch.Size([10, 128])
classifier.4.bias torch.Size([10])


In [76]:
summary(model = net, input_size = (3, 32, 32), batch_size = 1)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [1, 32, 32, 32]             896
       BatchNorm2d-2            [1, 32, 32, 32]              64
         LeakyReLU-3            [1, 32, 32, 32]               0
            Conv2d-4            [1, 32, 16, 16]           9,248
       BatchNorm2d-5            [1, 32, 16, 16]              64
         LeakyReLU-6            [1, 32, 16, 16]               0
            Conv2d-7            [1, 64, 16, 16]          18,496
       BatchNorm2d-8            [1, 64, 16, 16]             128
         LeakyReLU-9            [1, 64, 16, 16]               0
           Conv2d-10              [1, 64, 8, 8]          36,928
      BatchNorm2d-11              [1, 64, 8, 8]             128
        LeakyReLU-12              [1, 64, 8, 8]               0
           Linear-13                   [1, 128]         524,416
      BatchNorm1d-14                   

In [77]:
summary(model = net, input_size = (3, 32, 32), batch_size = 4)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [4, 32, 32, 32]             896
       BatchNorm2d-2            [4, 32, 32, 32]              64
         LeakyReLU-3            [4, 32, 32, 32]               0
            Conv2d-4            [4, 32, 16, 16]           9,248
       BatchNorm2d-5            [4, 32, 16, 16]              64
         LeakyReLU-6            [4, 32, 16, 16]               0
            Conv2d-7            [4, 64, 16, 16]          18,496
       BatchNorm2d-8            [4, 64, 16, 16]             128
         LeakyReLU-9            [4, 64, 16, 16]               0
           Conv2d-10              [4, 64, 8, 8]          36,928
      BatchNorm2d-11              [4, 64, 8, 8]             128
        LeakyReLU-12              [4, 64, 8, 8]               0
           Linear-13                   [4, 128]         524,416
      BatchNorm1d-14                   

## 训练 

In [106]:
import torch.optim as optim
from torch.autograd import Variable

In [107]:
optimizer = optim.Adam(params=net.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()

In [108]:
import time

In [112]:
running_loss = 0
start_time = time.time()
for epoch in range(2):
    for i, data in enumerate(train_loader):
        images, labels = data
        images, labels = Variable(images), Variable(labels)
        if torch.cuda.is_available():
            net.cuda()
            images, labels = images.cuda(), labels.cuda()
        outputs = net(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # 打印log信息
       
        running_loss += loss.data
        if i % 2000 == 1999:  # 每2000个batch打印一次训练信息
            print('[%d %5d] loss: %.3f  time: %.3fs' % (epoch+1, i+1, running_loss/2000, time.time()-start_time))
            running_loss = 0
            start_time = time.time()
print('Finish Training')

[1  2000] loss: 1.927  time: 16.891s
[1  4000] loss: 1.929  time: 14.305s
[1  6000] loss: 1.929  time: 14.242s
[1  8000] loss: 1.927  time: 14.063s
[1 10000] loss: 1.921  time: 15.137s
[1 12000] loss: 1.919  time: 16.550s
[2  2000] loss: 2.389  time: 18.905s
[2  4000] loss: 1.909  time: 15.857s
[2  6000] loss: 1.901  time: 16.594s
[2  8000] loss: 1.911  time: 15.210s
[2 10000] loss: 1.902  time: 14.481s
[2 12000] loss: 1.909  time: 16.202s
Finish Training


In [113]:
# 整个测试集上效果
total = 0  # 总共的图片数
correct = 0  # 正确的图片数
for images, labels in test_loader:
    outputs = net(Variable(images).cuda())
    _, predicted = torch.max(outputs, 1)
    total += predicted.size()[0]  # 加4
    correct += (predicted == labels.cuda()).sum()
print('correct: %d, total: %d' %(correct, total))
print('整个测试集中的准确率为 %.2f%%' % (100 * correct / total))

correct: 5451, total: 10000
整个测试集中的准确率为 54.51%
